# 📦 Data Extraction from RÚZ API

This notebook extracts structured financial statement data from the public API of the **Register účtovných závierok (RÚZ)**.

We focus on companies within a selected SK NACE classification over a range of years (2014–2023), using:
- Filters for entity type (e.g., a.s., s.r.o.)
- Year and industry
- Balance sheet and income statement formats

The extracted data will serve as a base for further analysis and model development.


In [ ]:
import requests
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor

#1.step is to export ICO (company identification number) - first api call will be based on these numbers
# List of companies including ICO can be downloaded by expanded search using our desired criteria
ico_xlsx_path = "ico.xlsx"
sheet_name = "ico"
ico_column = "ico" # -----IMPROVEMENTS - load ico directly from the downloaded file with list of companies...

output_muj_csv = "output_mikro.csv" #output file for micro enterprises
output_pod_csv = "output_small.csv" #output file for small enterprises - they have different schemas for reporting, therefore we must separate them

muj_header_csv = "uc_mikro.csv" # Header file micro enterprises
pod_header_csv = "uc_small.csv" # Header file small enterprises

# (OPTIONAL) HEADERS LOADING
header_muj = pd.read_csv(muj_header_csv, nrows=1).columns.tolist()
header_pod = pd.read_csv(pod_header_csv, nrows=1).columns.tolist()

# API CALL
def safe_request(url, delay=0.5):
    try:
        time.sleep(delay)
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Error {response.status_code}: {url}")
    except Exception as e:
        print(f"❌ Exception: {e} | {url}")
    return None

# EXTRACTION OF ROWS FROM FINANCIAL STATEMENTS
def extract_row_from_vykaz(vykaz: dict, header: list) -> dict:
    row = {}
    titulna = vykaz.get("obsah", {}).get("titulnaStrana", {})
    adresa = titulna.get("adresa", {})

    metadata = {
        "id": vykaz.get("id"),
        "idUctovnejZavierky": vykaz.get("idUctovnejZavierky"),
        "pristupnostDat": vykaz.get("pristupnostDat"),
        "datumPoslednejUpravy": vykaz.get("datumPoslednejUpravy"),
        "zdrojDat": vykaz.get("zdrojDat"),
        "kodDanovehoUradu": vykaz.get("kodDanovehoUradu"),
        "idSablony": vykaz.get("idSablony"),
        "nazovUctovnejJednotky": titulna.get("nazovUctovnejJednotky"),
        "ico": titulna.get("ico"),
        "dic": titulna.get("dic"),
        "skNace": titulna.get("skNace"),
        "typZavierky": titulna.get("typZavierky"),
        "obdobieOd": titulna.get("obdobieOd"),
        "obdobieDo": titulna.get("obdobieDo"),
        "predchadzajuceObdobieOd": titulna.get("predchadzajuceObdobieOd"),
        "predchadzajuceObdobieDo": titulna.get("predchadzajuceObdobieDo"),
        "datumSchvalenia": titulna.get("datumSchvalenia"),
        "datumZostavenia": titulna.get("datumZostavenia"),
        "oznacenieObchodnehoRegistra": titulna.get("oznacenieObchodnehoRegistra"),
        "adresa.ulica": adresa.get("ulica"),
        "adresa.cislo": adresa.get("cislo"),
        "adresa.psc": adresa.get("psc"),
        "adresa.mesto": adresa.get("mesto"),
    }

    row.update(metadata)

    tabulky = vykaz.get("obsah", {}).get("tabulky", [])
    data_values = []
    for tab in tabulky:
        data_values.extend(tab.get("data", []))

    data_keys = [key for key in header if key not in metadata]
    for i, key in enumerate(data_keys):
        value = data_values[i] if i < len(data_values) else ""
        row[key] = value if value not in [None, ""] else 0

    return row

# FULL LOOP OF ALL API CALLS
def process_ico(ico):
    print(f"Working on ICO: {ico}")
    jednotky = safe_request(f"https://www.registeruz.sk/cruz-public/api/uctovne-jednotky?zmenene-od=1000-01-01&ico={ico}") #First API call
    rows_muj = []
    rows_pod = []

    if jednotky and jednotky.get("id"):
        jednotka_id = jednotky["id"][0]
        print(f"ID of Company: {jednotka_id}")
        jednotka = safe_request(f"https://www.registeruz.sk/cruz-public/api/uctovna-jednotka?id={jednotka_id}") #Second API call
        if jednotka and jednotka.get("idUctovnychZavierok"):
            for zavierka_id in jednotka["idUctovnychZavierok"]:
                print(f"\n   ➡️ ID of Financial Statement: {zavierka_id}")
                zavierka = safe_request(f"https://www.registeruz.sk/cruz-public/api/uctovna-zavierka?id={zavierka_id}") #Third API call
                if not zavierka or not zavierka.get("idUctovnychVykazov"):
                    print(f"      ⚠️ Financial Statement {zavierka_id} has no reports.")
                    continue

                for vykaz_id in zavierka["idUctovnychVykazov"]:
                    print(f"      🔸 ID of report: {vykaz_id}")
                    vykaz = safe_request(f"https://www.registeruz.sk/cruz-public/api/uctovny-vykaz?id={vykaz_id}") #Fourth API call
                    if not vykaz:
                        continue
                    #Loop for filtering companies based on their size (MUJ = mikro, POD = small)
                    dokument_typ = "unknown"
                    for priloha in vykaz.get("attachments", []):
                        meno = priloha.get("name", "").upper()
                        print(f"         🧾 name of attachment: {meno}")
                        if "MUJ" in meno:
                            dokument_typ = "muj"
                            break
                        elif "POD" in meno:
                            dokument_typ = "pod"
                            break

                    if dokument_typ == "muj":
                        rows_muj.append(extract_row_from_vykaz(vykaz, header_muj))
                        print("         ✅ Added to MUJ")
                    elif dokument_typ == "pod":
                        rows_pod.append(extract_row_from_vykaz(vykaz, header_pod))
                        print("         ✅ Added to POD")
                    else:
                        print(f"         ⚠️ Unkown type of document for report {vykaz_id}")
    return rows_muj, rows_pod

# LOADING OF ICO
ico_df = pd.read_excel(ico_xlsx_path, sheet_name=sheet_name)
ico_list = ico_df[ico_column].dropna().astype(str).tolist()

# PARALLEL COMPUTING
rows_muj = []
rows_pod = []

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(process_ico, ico) for ico in ico_list]
    for future in futures:
        r_muj, r_pod = future.result()
        rows_muj.extend(r_muj)
        rows_pod.extend(r_pod)

# CSV FILLING
if rows_muj:
    pd.DataFrame(rows_muj).to_csv(output_muj_csv, index=False)
    print(f"\n✅ Micro companies saved to: {output_muj_csv}")

if rows_pod:
    pd.DataFrame(rows_pod).to_csv(output_pod_csv, index=False)
    print(f"✅ Small companies saved to: {output_pod_csv}")

if not rows_muj and not rows_pod:
    print("❌ No data has been retrieved.")
